In [1]:
!pip install tensorflow-io


In [2]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import polars as pl
import tensorflow_io as tfio

load_dotenv()

True

In [3]:
# setup database
client = MongoClient(os.environ["URI"])

# Get reference to 'bank' database
db = client.flowers

# Get a reference to 'magazines' collection
train_collection = db.train

train_df = pl.read_csv(os.path.join(os.environ["PATH_ARCHIVE"], "train.csv"))

In [4]:
train_df.head()

image:FILE,category
str,i64
"""train/carnatio…",0
"""train/carnatio…",0
"""train/carnatio…",0
"""train/carnatio…",0
"""train/carnatio…",0


In [5]:
# remove the paths from the df
train_df = train_df.with_columns(pl.col("image:FILE").str.split(by="/").alias("split"))
train_df = train_df.select(
    pl.col("split").list.get(2).str.strip_suffix("_c.jpg").alias("id"),
    pl.col("split").list.get(1).alias("flower_type"),
    pl.col("image:FILE").alias("filepath"),
    pl.col("category"),
)

In [6]:
train_df.to_dicts()

[{'id': '6838762136_f8254d6a1a',
  'flower_type': 'carnation',
  'filepath': 'train/carnation/6838762136_f8254d6a1a_c.jpg',
  'category': 0},
 {'id': '3540544866_823aee81af',
  'flower_type': 'carnation',
  'filepath': 'train/carnation/3540544866_823aee81af_c.jpg',
  'category': 0},
 {'id': '50034862576_b0f55b7b3b',
  'flower_type': 'carnation',
  'filepath': 'train/carnation/50034862576_b0f55b7b3b_c.jpg',
  'category': 0},
 {'id': '26770795691_f662e47143',
  'flower_type': 'carnation',
  'filepath': 'train/carnation/26770795691_f662e47143_c.jpg',
  'category': 0},
 {'id': '18224456534_2f75191d85',
  'flower_type': 'carnation',
  'filepath': 'train/carnation/18224456534_2f75191d85_c.jpg',
  'category': 0},
 {'id': '49557582587_a6c5056105',
  'flower_type': 'carnation',
  'filepath': 'train/carnation/49557582587_a6c5056105_c.jpg',
  'category': 0},
 {'id': '3311909785_beb861c848',
  'flower_type': 'carnation',
  'filepath': 'train/carnation/3311909785_beb861c848_c.jpg',
  'category': 0}

In [9]:
result = train_collection.insert_many(train_df.to_dicts())

document_ids = result.inserted_ids
print("# of documents inserted: " + str(len(document_ids)))
print(f"_ids of inserted documents: {document_ids}")

client.close()

# of documents inserted: 13642
_ids of inserted documents: [ObjectId('654d431c68873ce6318822e4'), ObjectId('654d431c68873ce6318822e5'), ObjectId('654d431c68873ce6318822e6'), ObjectId('654d431c68873ce6318822e7'), ObjectId('654d431c68873ce6318822e8'), ObjectId('654d431c68873ce6318822e9'), ObjectId('654d431c68873ce6318822ea'), ObjectId('654d431c68873ce6318822eb'), ObjectId('654d431c68873ce6318822ec'), ObjectId('654d431c68873ce6318822ed'), ObjectId('654d431c68873ce6318822ee'), ObjectId('654d431c68873ce6318822ef'), ObjectId('654d431c68873ce6318822f0'), ObjectId('654d431c68873ce6318822f1'), ObjectId('654d431c68873ce6318822f2'), ObjectId('654d431c68873ce6318822f3'), ObjectId('654d431c68873ce6318822f4'), ObjectId('654d431c68873ce6318822f5'), ObjectId('654d431c68873ce6318822f6'), ObjectId('654d431c68873ce6318822f7'), ObjectId('654d431c68873ce6318822f8'), ObjectId('654d431c68873ce6318822f9'), ObjectId('654d431c68873ce6318822fa'), ObjectId('654d431c68873ce6318822fb'), ObjectId('654d431c68873ce631

NotImplementedError: unable to open file: libtensorflow_io.so, from paths: ['c:\\FlowerClassification\\.venv\\flower\\Lib\\site-packages\\tensorflow_io\\python\\ops\\libtensorflow_io.so']
caused by: ['c:\\FlowerClassification\\.venv\\flower\\Lib\\site-packages\\tensorflow_io\\python\\ops\\libtensorflow_io.so not found']